Goal: Predict the probability by gender of getting hired in a company

Evaluation Criteria: Prioritise accuracy to determine the probability by gender of getting hired in a company 

Description: I will train a logistic regression to predict the probability of earning over
50K (“hired”) from the adult_reconstruction.csv dataset. Categorical features
will be one-hot encoded and numerical features standardized. The data will be
split 80/20 stratified by the target, and I will compute overall ROC AUC as the
evaluation metric. Finally, the script prints the AUC on the hold-out set.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Load data
df = pd.read_csv("./input/adult_reconstruction.csv")

# Define target and features
df = df.dropna(subset=["income", "gender"])
y = (df["income"].str.strip() == ">50K").astype(int)
X = df.drop(columns=["income"])

# Identify column types
numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()

# Preprocessing pipelines
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)
categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="constant", fill_value="Unknown")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("cat", categorical_transformer, categorical_cols),
    ]
)

# Split data
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Create and train model
clf = Pipeline(
    steps=[("preproc", preprocessor), ("clf", LogisticRegression(max_iter=1000))]
)
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred_proba = clf.predict_proba(X_val)[:, 1]
auc = roc_auc_score(y_val, y_pred_proba)
print(f"ROC AUC on validation set: {auc:.4f}")

AttributeError: Can only use .str accessor with string values!